In [1]:
import pandas as pd

avaliacoes = {
    'Ana': {
        'Freddy x Jason': 2.5, 
        'O Ultimato Bourne': 3.5,
        'Star Trek': 3.0, 
        'Exterminador do Futuro': 3.5, 
        'Norbit': 2.5, 
        'Star Wars': 3.0
    },
    'Marcos': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 3.5, 
        'Star Trek': 1.5, 
        'Exterminador do Futuro': 5.0, 
        'Star Wars': 3.0, 
        'Norbit': 3.5
    },
    'Pedro': {
        'Freddy x Jason': 2.5, 
        'O Ultimato Bourne': 3.0,
        'Exterminador do Futuro': 3.5, 
        'Star Wars': 4.0
    },
    'Claudia': {
        'O Ultimato Bourne': 3.5, 
        'Star Trek': 3.0,
        'Star Wars': 4.5, 
        'Exterminador do Futuro': 4.0, 
        'Norbit': 2.5
    },
    'Adriano': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 4.0, 
        'Star Trek': 2.0, 
        'Exterminador do Futuro': 3.0, 
        'Star Wars': 3.0,
        'Norbit': 2.0
    },
    'Janaina': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 4.0,
        'Star Wars': 3.0, 
        'Exterminador do Futuro': 5.0, 
        'Norbit': 3.5
    },
    'Leonardo': {
        'O Ultimato Bourne': 4.5,
        'Norbit': 1.0,
        'Exterminador do Futuro': 4.0
    }
}

df = pd.DataFrame(avaliacoes).transpose()
df

,Freddy x Jason,O Ultimato Bourne,Star Trek,Exterminador do Futuro,Norbit,Star Wars
Ana,2.5,3.5,3.0,3.5,2.5,3.0
Marcos,3.0,3.5,1.5,5.0,3.5,3.0
Pedro,2.5,3.0,NaN,3.5,NaN,4.0
Claudia,NaN,3.5,3.0,4.0,2.5,4.5
Adriano,3.0,4.0,2.0,3.0,2.0,3.0
Janaina,3.0,4.0,NaN,5.0,3.5,3.0
Leonardo,NaN,4.5,NaN,4.0,1.0,NaN


In [2]:
import math
import pandas as pd

def euclidiana(Data, usuario1, usuario2):
    si = {}
    for item in Data.columns:
        if not (pd.isna(Data.at[usuario1, item]) or pd.isna(Data.at[usuario2, item])):
            si[item] = 1

    if len(si) == 0:
        return 0

    soma = sum(
        pow(Data.at[usuario1, item] - Data.at[usuario2, item], 2)
        for item in si
    )

    return 1 / (1 + math.sqrt(soma))

In [3]:
euclidiana(df,'Claudia', 'Ana')

0.38742588672279304

In [4]:
def get_similaridade(Data, usuario):
    similaridade = [(euclidiana(Data, usuario, outro), outro)
                     for outro in Data.index if outro != usuario]
    
    similaridade.sort(reverse=True)
    
    # Retorna os 30 mais similares
    return similaridade[:30]

In [5]:
get_similaridade(df, 'Leonardo')

[(0.4, 'Adriano'),
 (0.38742588672279304, 'Pedro'),
 (0.3567891723253309, 'Claudia'),
 (0.3483314773547883, 'Ana'),
 (0.2674788903885893, 'Janaina'),
 (0.25824569976124334, 'Marcos')]

In [12]:
def get_recomendacao(Data, usuario, nota_minima=2.0):
    similaridades = []

    # Calcular similaridade com todos os outros usuários
    for outro_usuario in Data.index:
        if outro_usuario == usuario:
            continue

        similaridade = euclidiana(Data, usuario, outro_usuario)
        similaridades.append((outro_usuario, similaridade))

    # Ordenar por similaridade em ordem decrescente
    similaridades.sort(key=lambda x: x[1], reverse=True)

    # Escolher o usuário mais similar (o primeiro da lista)
    usuario_mais_similar = similaridades[0][0]

    # Filtrar os filmes que o usuário mais similar assistiu e o usuário atual ainda não assistiu, considerando a nota mínima
    filmes_recomendados = Data.loc[usuario_mais_similar][
        (Data.loc[usuario].isna()) & (Data.loc[usuario_mais_similar] >= nota_minima)
    ].index.tolist()

    
    return filmes_recomendados[:30]

In [7]:
get_recomendacao(df, 'Claudia')

['Freddy x Jason']

In [8]:
def carrega_dados(path='C:\\Users\\diham\\Downloads\\ml-100k'):
    filmes = {}
    for linha in open(path + '/u.item', encoding='latin-1'):
        (id, titulo) = linha.split('|')[:2]
        filmes[id] = titulo
    base = {}
    for linha in open(path + '/u.data', encoding='latin-1'):
        (usuario, id_filme, nota, tempo) = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][filmes[id_filme]] = float(nota)

    return base

In [9]:
data = carrega_dados()
data = pd.DataFrame(data).T
data.head()

,Kolya (1996),Mrs. Doubtfire (1993),Muriel's Wedding (1994),Shall We Dance? (1996),Stand by Me (1986),Ace Ventura: Pet Detective (1994),"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)",Raising Arizona (1987),Being There (1979),"Truth About Cats & Dogs, The (1996)",...,Kika (1993),Mirage (1995),Mamma Roma (1962),"War at Home, The (1996)","Sunchaser, The (1996)",Sweet Nothing (1995),B. Monkey (1998),Mat' i syn (1997),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
196,3.0,4.0,4.0,3.0,5.0,5.0,4.0,4.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,3.0,4.0,NaN,5.0,4.0,NaN,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
get_similaridade(data, "1")

[(1.0, '812'),
 (1.0, '418'),
 (1.0, '155'),
 (0.5, '729'),
 (0.5, '631'),
 (0.5, '351'),
 (0.5, '309'),
 (0.5, '273'),
 (0.4142135623730951, '876'),
 (0.4142135623730951, '485'),
 (0.4142135623730951, '111'),
 (0.36602540378443865, '687'),
 (0.36602540378443865, '105'),
 (0.3333333333333333, '895'),
 (0.3333333333333333, '811'),
 (0.3333333333333333, '685'),
 (0.3333333333333333, '531'),
 (0.3333333333333333, '39'),
 (0.3333333333333333, '356'),
 (0.3333333333333333, '341'),
 (0.3333333333333333, '282'),
 (0.3333333333333333, '260'),
 (0.3333333333333333, '107'),
 (0.3090169943749474, '9'),
 (0.3090169943749474, '842'),
 (0.3090169943749474, '820'),
 (0.3090169943749474, '696'),
 (0.3090169943749474, '547'),
 (0.3090169943749474, '520'),
 (0.3090169943749474, '516')]

In [11]:
get_recomendacao(data, "211")

['Mrs. Brown (Her Majesty, Mrs. Brown) (1997)',
 'Murder at 1600 (1997)',
 'Cop Land (1997)',
 'Scream (1996)',
 'Beautician and the Beast, The (1997)',
 'Kiss the Girls (1997)',
 'Liar Liar (1997)',
 'Edge, The (1997)',
 'Lost Highway (1997)',
 'Conspiracy Theory (1997)',
 "Dante's Peak (1997)",
 'Saint, The (1997)',
 "Devil's Own, The (1997)",
 'G.I. Jane (1997)',
 'Mother (1996)',
 'Crash (1996)',
 'Love Jones (1997)',
 'Everyone Says I Love You (1996)',
 'Rosewood (1997)',
 'Anna Karenina (1997)']